In [1]:
import datetime
import math
from prettytable import PrettyTable

#=========================================================================
# Team members: Woolim Lee, Chuandong Liu
#=========================================================================
# a menu item, followed by the respective cookie mix
menu = {  'Delicious Duo' : {'Sugar' : 3, 'Chocolate' : 3, 'Peanut' : 0 }, 
               'Six Pack' : {'Sugar' : 2, 'Chocolate' : 2, 'Peanut' : 2 }, 
            'Sugar Snack' : {'Sugar' : 6, 'Chocolate' : 0, 'Peanut' : 0 },
         'Burdell Bundle' : {'Sugar' : 5, 'Chocolate' : 5, 'Peanut' : 5 } }

# individual prices for each cookie type
cookiePrices = {'Sugar' : 1.00, 'Chocolate' : 2.00, 'Peanut' : 3.00}

# current cookie inventory for each cookie type
cookieInv    = {'Sugar' : 40,   'Chocolate' : 25,   'Peanut' : 13}

# past orders (in reality, would be in a DBMS)
'Legend: SS => Sugar Snack; SP => Six Pack; DD => Delicious Duo; BB => Burdell Bundle'
'Content:   tranCode, orderQty, orderDateTime, tranCost'

orderRec = { 'SS1' : {5, '2022-02-26 13:52', 33.00 }, 
             'BB1' : {2, '2022-02-26 15:13', 66.00 },
             'DD1' : {3, '2022-02-26 15:32', 29.70 },
             'SP1' : {1, '2022-02-26 15:43', 13.20 },
             'DD2' : {6, '2022-02-26 16:03', 59.40 },
             'BB2' : {3, '2022-02-26 16:12', 99.00 },
             'DD3' : {4, '2022-02-26 16:14', 39.60 }         
           }
#===========================functions====================================
def displayMenu(Amenu):
    sorted_menu = sorted(Amenu.items())
    sorted_menu = list(sorted_menu)
    
    x = PrettyTable()
    x.field_names = ['Menu Items','Sugar Cookie','Chocolate Cookie','Peanut Cookie','Menu Item Price']
    for item in sorted_menu:
        row=[]
        item = list(item)
        row = []
        key = item[0]
        totalprice = getMenuPrice(key)
        
        row.append(key)
        for quant in item[1].values():
            row.append(quant)
        row.append(totalprice)
        
        x.add_row(row)
        
        row=[]
        continue
    print(x)
    
def getMenuPrice(menuItem):
    Sugar_price = cookiePrices['Sugar']
    Choco_price = cookiePrices['Chocolate']
    Peanut_price = cookiePrices['Peanut']
    grandtotal = 0
    
    for cookie,quantity in menu[menuItem].items():
        if cookie == 'Sugar':
            grandtotal += quantity * Sugar_price
        elif cookie == 'Chocolate':
            grandtotal += quantity * Choco_price
        elif cookie == "Peanut":
            grandtotal += quantity * Peanut_price
            
    return grandtotal


def maxUnitsCanOrder(menu, chosenMenuItem, cookieInv):
    demand = list(menu[chosenMenuItem].values())
    supply = list(cookieInv.values())
    capacity = []
    for i in range (len(supply)):
        if demand[i] != 0:
            capacity.append(supply[i]/demand[i])
            
    #capacity = [x/y for x in supply for y in demand if supply.index(x) == demand.index(y) if y !=0]
    return math.floor(min(capacity))

def decreaseInv(menu, chosenMenuItem, cookieInv, orderQty):
    Item = menu[chosenMenuItem]
    for cookieType in Item:
        cookieInv[cookieType] = cookieInv[cookieType] - orderQty * Item[cookieType]

def calcTranCost(chosenMenuItem, orderQty):
    price_of_item = getMenuPrice(chosenMenuItem)
    total_cost = price_of_item * float(orderQty)
    total_w_tax = total_cost +(total_cost *0.1)
    return total_w_tax


def recordOrder(orderRec, chosenMenuItem, orderQty, tranCost):
    tranCodes = list(orderRec.keys())
    if chosenMenuItem == 'Sugar Snack':
        legend = 'SS'
    elif chosenMenuItem == 'Six Pack':
        legend = 'SP'
    elif chosenMenuItem == 'Delicious Duo':
        legend = 'DD'
    elif chosenMenuItem == 'Burdell Bundle':
        legend = 'BB'

    position = 0
    for code in tranCodes:
        if legend in code:
            if int(code[2]) > position:
                position = int(code[2])
    position += 1
    tranCode = legend + str(position)
    now = datetime.datetime.now()
    dt_string = now.strftime("%Y-%m-%d %H:%M")  ##2022-02-26 16:14
    orderRec[tranCode] = {orderQty, dt_string, tranCost}

def order(): #this is not a required function but a function to interact with customer
    divider = '---------------------------------'
    welcome = "Welcome to Hot Byte Cookies! \nHere's our yummy menu!"
    order = "Let's take your order "

    print(welcome)
    displayMenu(menu)
    print(order)

    menu_item =input('From our menu, please enter the Menu Item you want to buy: ')
    item_quantity = int(input('Please enter the number of '+ menu_item + ' you want to buy:'))

    review_order = 'You have chosen to order ' + str(item_quantity) +  ' boxes of menu item: ' + menu_item
    print(review_order)

    if item_quantity <= maxUnitsCanOrder(menu, menu_item, cookieInv):
        transaction_total = calcTranCost(menu_item,item_quantity)
        recordOrder(orderRec,menu_item,item_quantity, calcTranCost(menu_item,item_quantity))
        decreaseInv(menu,menu_item, cookieInv, item_quantity)
        transaction_comment = 'Your total cost for ' + str(item_quantity) +' '+ menu_item + ' boxes will be $' + str(transaction_total) + '. Time to pay up!\nThanks for your business. See you next time!'
        print('Yes we can do that! Hot cookies coming up. Hang tight!')
        print(transaction_comment)
    else:
        print("So sorry!! We've had a run on cookies and can only cover " +  str(maxUnitsCanOrder(menu, menu_item, cookieInv)) + " " + menu_item + ". Are we good to go with this quantity?")
        answer = input('Yes or No?: ')
        if answer == 'Yes':
            transaction_total = calcTranCost(menu_item,maxUnitsCanOrder(menu,menu_item, cookieInv))
            print("Super! Cookies coming up. I have to say, you are looking terrific today!")
            transaction_comment = 'Your total cost for ' + str(maxUnitsCanOrder(menu, menu_item, cookieInv)) +' '+ menu_item + ' boxes will be $' + str(transaction_total) + '. Time to pay up!\nThanks for your business. See you next time!'
            recordOrder(orderRec,menu_item,maxUnitsCanOrder(menu, menu_item, cookieInv), transaction_total)
            decreaseInv(menu,menu_item, cookieInv, maxUnitsCanOrder(menu, menu_item, cookieInv))
            print(transaction_comment)
        else:
            print("Well don't be a stranger! We'll expect to see you back sometime soon!")


    print(divider)
    print ('Updated orderRec ==>',orderRec)
    print(divider)
    print('Ending Inventory ==>', cookieInv) 

#=========================================================================            
print('TESTING SCENARIO #1 \n')
order()

TESTING SCENARIO #1 

Welcome to Hot Byte Cookies! 
Here's our yummy menu!
+----------------+--------------+------------------+---------------+-----------------+
|   Menu Items   | Sugar Cookie | Chocolate Cookie | Peanut Cookie | Menu Item Price |
+----------------+--------------+------------------+---------------+-----------------+
| Burdell Bundle |      5       |        5         |       5       |       30.0      |
| Delicious Duo  |      3       |        3         |       0       |       9.0       |
|    Six Pack    |      2       |        2         |       2       |       12.0      |
|  Sugar Snack   |      6       |        0         |       0       |       6.0       |
+----------------+--------------+------------------+---------------+-----------------+
Let's take your order 
From our menu, please enter the Menu Item you want to buy: Six Pack
Please enter the number of Six Pack you want to buy:5
You have chosen to order 5 boxes of menu item: Six Pack
Yes we can do that! Hot cook

In [2]:
# Testing Scenario #2

TESTING SCENARIO #2 

Welcome to Hot Byte Cookies! 
Here's our yummy menu!
+----------------+--------------+------------------+---------------+-----------------+
|   Menu Items   | Sugar Cookie | Chocolate Cookie | Peanut Cookie | Menu Item Price |
+----------------+--------------+------------------+---------------+-----------------+
| Burdell Bundle |      5       |        5         |       5       |       30.0      |
| Delicious Duo  |      3       |        3         |       0       |       9.0       |
|    Six Pack    |      2       |        2         |       2       |       12.0      |
|  Sugar Snack   |      6       |        0         |       0       |       6.0       |
+----------------+--------------+------------------+---------------+-----------------+
Let's take your order 
From our menu, please enter the Menu Item you want to buy: Sugar Snack
Please enter the number of Sugar Snack you want to buy:8
You have chosen to order 8 boxes of menu item: Sugar Snack
So sorry!! We've ha

In [3]:
# Testing Scenario #3

TESTING SCENARIO #3 

Welcome to Hot Byte Cookies! 
Here's our yummy menu!
+----------------+--------------+------------------+---------------+-----------------+
|   Menu Items   | Sugar Cookie | Chocolate Cookie | Peanut Cookie | Menu Item Price |
+----------------+--------------+------------------+---------------+-----------------+
| Burdell Bundle |      5       |        5         |       5       |       30.0      |
| Delicious Duo  |      3       |        3         |       0       |       9.0       |
|    Six Pack    |      2       |        2         |       2       |       12.0      |
|  Sugar Snack   |      6       |        0         |       0       |       6.0       |
+----------------+--------------+------------------+---------------+-----------------+
Let's take your order 
From our menu, please enter the Menu Item you want to buy: Burdell Bundle
Please enter the number of Burdell Bundle you want to buy:3
You have chosen to order 3 boxes of menu item: Burdell Bundle
So sorry!!